In [15]:
import glob
import os
import io
import json
import shutil
from PIL import Image
img_dir='/home/kiki/dados_servidor/Servidor/Teses/deepsearch_dataset/images/'
bboxes_dir='/home/kiki/dados_servidor/Servidor/Teses/deepsearch_dataset/bboxes/'

yolo_images_folder='/home/kiki/dados_servidor/Servidor/Teses/labeling/yolo_data/obj_train_data'
yolo_labels_folder='/home/kiki/dados_servidor/Servidor/Teses/labeling/yolo_data/obj_train_data'

#convert images,coordinates and classes to a yolo dataset format
def to_yolo(images_paths,coordinates,classes,yolo_images_folder,yolo_labels_folder):
    for image in images_paths:
        shutil.copy(image,yolo_images_folder)
    for i in range(len(images_paths)):
        with io.open(yolo_labels_folder+'/'+images_paths[i].split('/')[-1].split('.')[0]+'.txt','w',encoding='utf8') as f:
            for j in range(len(coordinates[i])):
                f.write(str(classes[i][j])+' '+str(coordinates[i][j][0])+' '+str(coordinates[i][j][1])+' '+str(coordinates[i][j][2])+' '+str(coordinates[i][j][3])+'\n')

images_folders=os.listdir(img_dir)[:1]

images_paths = []
coordinates=[]
classes=[]

def convert(size, box):
    dw = 1./size[0]
    dh = 1./size[1]
    x = (box[0] + box[1])/2.0
    y = (box[2] + box[3])/2.0
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)

labels_dict={'title':1,'paragraph':2,'picture':3,'table':4,'formula':5,'footnote':6,'page-footer':7,'page-header':8,'subtitle-level-1':9,'caption':10}
resolution=150
for folder in images_folders:
    annotations=json.load(open((bboxes_dir+folder+'.json')))
    for annotation in annotations.keys():
        img_path=img_dir+folder+'/'+folder+'_'+annotation+'.jpg'
        images_paths.append(img_path)
        im=Image.open(img_path)
        w= int(im.size[0])
        h= int(im.size[1])
        bboxes=[an['bbox'] for an in annotations[annotation]]
        bboxes = [[x * resolution / 72 for x in b] for b in bboxes]
        for i in range(len(bboxes)):
            bbox=bboxes[i]
            x_min,y_min,x_max,y_max=int(bbox[0]),int(bbox[1]),int(bbox[2]),int(bbox[3])
            #convert to yolo format (x_center,y_center,width,height)
            bbox=convert((w,h),(x_min,x_max,y_min,y_max))
            bboxes[i]=bbox
        
        coordinates.append(bboxes)
        # classes.append([an['type'] for an in annotations[annotation]])
        classes.append([labels_dict[an['type']] for an in annotations[annotation]])
        
    to_yolo(images_paths,coordinates,classes,yolo_images_folder,yolo_labels_folder)

#obj names
with io.open('/home/kiki/dados_servidor/Servidor/Teses/labeling/yolo_data/obj.names','w',encoding='utf8') as f:
    c=0
    for key in labels_dict.keys():
        # f.write(key+'\n')
        f.write('label_'+str(c)+'\n')
        c+=1
    f.write('label_'+str(c)+'\n')
#train txt
with io.open('/home/kiki/dados_servidor/Servidor/Teses/labeling/yolo_data/train.txt','w',encoding='utf8') as f:
    for image in images_paths:
        f.write('data/obj_train_data/'+image.split('/')[-1]+'\n')
#obj data
with io.open('/home/kiki/dados_servidor/Servidor/Teses/labeling/yolo_data/obj.data','w',encoding='utf8') as f:
    f.write('classes = '+str(len(labels_dict.keys())+1)+'\n')
    f.write('train = data/train.txt\n')
    # f.write('valid = data/train.txt\n')
    f.write('names = data/obj.names\n')
    f.write('backup = backup/\n')

In [8]:
annotations

{'1': [{'bbox': [122.3088226318359,
    104.88976759765626,
    475.21997,
    194.3451508984375],
   'bbox_coords': [122.3088226318359,
    647.5748291015625,
    475.21997,
    737.0302124023438],
   'type': 'title'},
  {'bbox': [212.1371154785156,
    327.2694673046875,
    385.39001,
    341.88131422851563],
   'bbox_coords': [212.1371154785156,
    500.0386657714844,
    385.39001,
    514.6505126953125],
   'type': 'paragraph'},
  {'bbox': [87.1494369506836,
    453.09335036132813,
    510.3360900878906,
    487.8305634960938],
   'bbox_coords': [87.1494369506836,
    354.0894165039062,
    510.3360900878906,
    388.8266296386719],
   'type': 'title'},
  {'bbox': [259.8818969726562, 702.08798, 337.51001, 730.58453078125],
   'bbox_coords': [259.8818969726562, 111.33544921875, 337.51001, 139.832],
   'type': 'subtitle-level-1'}],
 '2': [{'bbox': [212.5029907226562,
    111.29143996093751,
    385.39001,
    126.28985304687501],
   'bbox_coords': [212.5029907226562,
    715.630126